In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing all necessary packages

In [ ]:
from sklearn.linear_model import LinearRegression
import sklearn.model_selection
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn.metrics
import pylab as plt
import scipy
from scipy import stats
import seaborn as sns
sns.set_palette("bright")
%matplotlib inline

In [ ]:
df = pd.read_csv (r'/kaggle/input/used-car-dataset-ford-and-mercedes/audi.csv')
print(df)

As I do not know what does 'tax' means in that dataset, I will delete it

In [ ]:
df.drop('tax', inplace=True, axis=1)

Basic descriptive statistics

In [ ]:
df.describe()

Checking types of different variables

In [ ]:
df.dtypes

Should convert model, year, transmission and fuelType to 'category' type

In [ ]:
df['year'] = df.year.astype('category')
df['model'] = df.model.astype('category')
df['transmission'] = df.transmission.astype('category')
df['fuelType'] = df.fuelType.astype('category')

In [ ]:
df.dtypes

Check it there any ommited variables

In [ ]:
df[df.isnull().any(axis=1)]

There is no such variables, great!

# Preparing numeric variables, outliers detection

In [ ]:
numeric_df = df[['price','mileage', 'mpg', 'engineSize']].copy()

In [ ]:
z_scores = stats.zscore(numeric_df)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
new_df = df[filtered_entries].copy()

In [ ]:
new_df.describe()

In [ ]:
for i in ['mileage','mpg', 'engineSize']:
  fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=True)
  fig.suptitle(i)
  sns.scatterplot(ax=axes[0], data=df, x="price", y=i)
  axes[0].set_title('before deleting outliers')
  sns.scatterplot(ax=axes[1], data=new_df, x="price", y=i)
  axes[1].set_title('after deleting outliers')
  
  
  sns.scatterplot(data=df, x="price", y=i)
  sns.scatterplot(data=new_df, x="price", y=i)
  axes[2].set_title('on one plot')

In [ ]:
print(len(new_df), len(df))

We can see that we deleted approximately 500 observations which is not critical for 10 hundred obresvations.

In [ ]:
for i in ['mileage','mpg', 'engineSize']:
  sns.set_style("whitegrid")
  fig, axes = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
  fig.suptitle(i)
  sns.boxplot(ax=axes[0], data=df, x=i)
  axes[0].set_title('before deleting outliers')
  sns.boxplot(ax=axes[1], data=new_df, x=i)
  axes[1].set_title('after deleting outliers')

In [ ]:
for i in ['price', 'mileage','mpg', 'engineSize']:
  fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(20, 5))
  fig.suptitle(i)
  ax1.hist(df[i])
  ax1.set_title('before deleting outliers')
  ax2.hist(new_df[i])
  ax2.set_title('after deleting outliers')

In [ ]:
for i in ['price', 'mileage','mpg', 'engineSize']:
  sns.set_style("whitegrid")
  fig, axes = plt.subplots(1, 2, figsize=(20, 5), sharey=True)
  fig.suptitle(i)
  sns.distplot(df[i], ax=axes[0], hist=True, kde=True, bins=int(180/5), hist_kws={'edgecolor':'black'}, kde_kws={'linewidth': 4})
  axes[0].set_title('before deleting outliers')
  sns.distplot(new_df[i], ax=axes[1], hist=True, kde=True, bins=int(180/5), hist_kws={'edgecolor':'black'}, kde_kws={'linewidth': 4})
  axes[1].set_title('after deleting outliers')

In [ ]:
new_df['log_price'] = np.log(new_df['price']) 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))
fig.suptitle('price')
ax1.hist(new_df['price'])
ax1.set_title('before log')
ax2.hist(new_df['log_price'])
ax2.set_title('after log')

In [ ]:
new_df.head()

In [ ]:
cols = new_df.columns.tolist()

cols = ['log_price', 'engineSize', 'mileage', 'mpg', 'model', 'year', 'transmission', 'fuelType']

new_df = new_df[cols]

new_df.head()

In [ ]:
# Correlations of features with target variable
correlations = new_df.corr(method='kendall')
correlations['log_price']

In [ ]:
# Plot correlation matrix
fig_1 = plt.figure(figsize=(12, 10))
new_correlations = new_df.corr(method='kendall')
sns.heatmap(new_correlations, annot=True, cmap='Greens', annot_kws={'size': 8})
plt.title('Correlation Matrix')
plt.show()

# Categorical variables preparation

In [ ]:
new_df["model_cat"] = new_df["model"].cat.codes
new_df["year_cat"] = new_df["year"].cat.codes
new_df["transmission_cat"] = new_df["transmission"].cat.codes
new_df["fuelType_cat"] = new_df["fuelType"].cat.codes

# Linear regression

# # Without categorical variables

In [ ]:
# Create X, y 
X = new_df.iloc[:, 1:4]
ones = np.ones([len(new_df), 1])
X = np.concatenate((ones, X), axis=1)
y = new_df.iloc[:, 0:1].values

model = sm.OLS(y, X)

results = model.fit()

In [ ]:
print(results.summary(yname='log_price', xname = ['const', 'engineSize', 'mileage', 'mpg']))

# # With categorical variables

In [ ]:
cols_2 = new_df.columns.tolist()

cols_2 = ['log_price', 'mileage', 'mpg', 'engineSize', 'model_cat', 'year_cat', 'transmission_cat', 'fuelType_cat']

new_df2 = new_df[cols_2].copy()

In [ ]:
cols_2 = new_df.columns.tolist()

cols_2 = ['log_price', 'mileage', 'mpg', 'engineSize', 'model_cat', 'year_cat', 'transmission_cat', 'fuelType_cat']

new_df2 = new_df[cols_2].copy()

# Create X, y 
X_2 = new_df2.iloc[:, 1:].values
ones = np.ones([len(new_df2), 1])
X_2 = np.concatenate((ones, X_2), axis=1)
y = new_df2.iloc[:, 0:1].values

reg_2 = LinearRegression().fit(X_2, y)

model2 = sm.OLS(y, X_2)

results2 = model2.fit()

In [ ]:
print(results2.summary(yname='log_price', xname = ['const', 'mileage', 'mpg', 'engineSize', 'model_cat', 'year_cat', 'transmission_cat', 'fuelType_cat']))

# Test and train sets

In [ ]:
X_2 = new_df2.iloc[:, 1:].values
ones = np.ones([len(new_df2), 1])
X_2 = np.concatenate((ones, X_2), axis=1)
y = new_df2.iloc[:, 0:1].values

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_2, y, test_size=0.4, random_state=0)

model3 = sm.OLS(y_train, X_train)

results3 = model3.fit()

yhat = results3.predict(X_test)

In [ ]:
mae = sklearn.metrics.mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)